<center style><h1>Computer Assignment 3</h1></center>
<center style><h1>Naive Bayes Classification</h1></center>

**In this assignment we are about to analyze some *divar.ir* ads and predict each ad category using naive bayes
classifier.**

We have ads in Vehicles, Electronic Devices, Business, For the Home, Personal, and Leisure & Hobbies.

## PreProcess

We will use *hazm* library to process persian text in our dataset.

In this preprocess we remove stop words and punctuation then use *hazm* to normalize and adjust spaces on title and
description. Then we divide sentences into words and find word root, so we can process them independently.

1 - In our text we might see a word in different forms, but they are from same root and can have same effect in our
classification. Stemming will remove prefix and postfix of a word to find it's root, but Lemmatization except removing
postfix and prefix of word, it will verify that root to be correct in meaning.

In our case Lemmatization has better performance and increase accuracy.

In [1]:
import pandas
from hazm import *

vehicle_frequency = {}
electronic_frequency = {}
business_frequency = {}
home_frequency = {}
personal_frequency = {}
hobbies_frequency = {}

category = ['vehicles', 'electronic-devices', 'businesses', 'for-the-home', 'personal', 'leisure-hobbies']
frequency = [vehicle_frequency, electronic_frequency, business_frequency, home_frequency, personal_frequency,
             hobbies_frequency]
stop_word = (stopwords_list() + ['.', '،', '؛', '[', ']', '{', '}', '(', ')', '?', '!', ':', '\"', '\'', '*', '/', '+',
                                 '%', '#',
                                 '-', '_', '\r\n', '\n', '«', '»', '۹', '۸', '۷', '۶', '۵', '۴', '۳', '۲', '۱', '۰'])

train_data = pandas.read_csv('Data/divar_train.csv')
test_data = pandas.read_csv('Data/divar_test.csv')

normalizer = Normalizer()
lemmatizer = Lemmatizer()
tokenizer = WordTokenizer()


def text_pre_process(sentence):
    text = normalizer.normalize(sentence)
    text = tokenizer.tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in text if not (word in stop_word)]
    return words


def pre_process(data):
    for i in range(len(data)):
        data['title'][i] = text_pre_process(data['title'][i])
        data['description'][i] = text_pre_process(data['description'][i])
    return data


train_data = pre_process(train_data)
train_data

,title,description,categories
0,"[بلبل, خرما]","[عدد, بلبل, خرما, ماهه, جوجه, خودم, بزرگشون, د...",leisure-hobbies
1,"[عینک, اسکی]","[عینک, اسکی, دبل, لنز, مارک, یو, وکس, نو, اصل,...",leisure-hobbies
2,"[تکیه, سر, تویوتا, پرادو]","[پارچه, سالم, تمیز]",vehicles
3,"[مجسمه, کریستال, ۲۴, ٪]","[مجسمه, دکوری, کریستال, بالرین, Rcr, ۲۴, ٪]",for-the-home
4,"[کیف, ساک]",[تمیز],personal
...,...,...,...
10195,"[ان, هاش, ۸۵]","[چی, شرط, خشک, ۲۶۰, کار]",vehicles
10196,"[۴۰۵, دوگانه, کارخانه, تمیز]","[فابریک, حلقه, لاستیک, ۲۰۵, نو, بیمه, یکسال, ت...",vehicles
10197,"[بخاری, گازی, دودکش, پلار]","[بخاری, نو, تمیز, هستش, مشتری, واقعی, میام]",for-the-home
10198,"[نر, کله, برنجی, چتر]","[سلام, دلیل, کمبود, واسباب, کشید#کش, کمترین, ق...",leisure-hobbies


In [2]:
test = pre_process(test_data)
test

,title,description,categories
0,"[کیف, مجلس, نو]","[کیف, مجلس, نو, نو]",personal
1,"[دیوار, کوب, نمد, تزیین]","[جهاز, عروس]",for-the-home
2,"[تیکه, تمیز, سالم]","[کشو, مخفی, شیک]",for-the-home
3,[مودم],"[سلام, مودم, سالم, وسایل, دیگه, ب, کاشت#کار, ن...",electronic-devices
4,"[۱۸, عددبوقلمون, قیمت, عمده]","[سلام, تعدای, بوقلمون, داشت#دار, جابه, فروش, ج...",leisure-hobbies
...,...,...,...
1795,"[باسکول, ۳۰۰, کیلویی]","[سالمه, سالمه, بشرط, قیمت, نوش, داخل, بازار, ۶...",businesses
1796,"[میز, مدیریت, کنفرانس]","[میز, مدیریت, کنفرانس, تمیز, سالم, خط, خش]",businesses
1797,"[باند, aiwa]","[سالم, اسپیکر]",electronic-devices
1798,"[عروس, هلند, مولد]","[عروس, هلند, جفت, ویک, تک, یه, جفت, دم, تخمه, ...",leisure-hobbies
